# PROJECT 3 :

### Author : Omer Ozeren 

## Introduction

Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

#### Import Packages

In [4]:
import nltk
from nltk.corpus import names
import pandas as pd
import random
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\OMERO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\names.zip.


True

I am going to use Naive Bayes as a Classifier to build a name gender classifier.I am going to use several different feature engineering process to select feasible features to be implemented in Naive Bayes model.

### Data Collection

In [5]:
# load names
labeled_names = ([(name, "male") for name in names.words("male.txt")] + 
                 [(name, "female") for name in names.words("female.txt")])

# random shuffle the names
random.shuffle(labeled_names)

I will begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev-test set, and the remaining 6900 words for the training set. 

In [6]:
# split data into three subsets
data_train = labeled_names[1500:]
data_devtest = labeled_names[500:1000]
data_test = labeled_names[:500]

**I also need to create a function to count error which presents variance between actual and model prediction.**

In [ ]:
def error(gender_features):
    # error list
    errors = []
    # extract mislabels
    for (name, tag) in devtest_names:
        guess = classifier.classify(gender_features(name))
        if guess != tag:
            errors.append((tag, guess, name))
    print("Number of Errors: ", len(errors))
    # print the errors
    for (tag, guess, name) in sorted(errors):
        print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))